In [59]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from IPython.display import clear_output
stopwords = pd.read_pickle('/Users/chinmayshrivastava/Desktop/OSCA/V4.0/stopwords.pickle')
from nltk import everygrams

In [81]:
headings = pd.read_pickle('/Users/chinmayshrivastava/Desktop/OSCA/V4.0/scraping/headingsfromhnandbc101.pickle')
paragraphs = pd.read_pickle('/Users/chinmayshrivastava/Desktop/OSCA/V4.0/scraping/paragraphsfromhnandbc101.pickle')
kws = pd.read_pickle('kws.pickle')

In [82]:
len(kws)

6037

In [83]:
toadd = []
for kw in kws:
    grams = everygrams(kw.split(' '))
    for gram in grams:
        toadd.append(' '.join(list(gram)))
kws = {}
kwid = 0
for element in toadd:
    if element not in kws:
        kws[element] = kwid
        kwid+=1

In [84]:
len(kws)

6252

In [85]:
hvector = CountVectorizer(vocabulary=kws, ngram_range=(1, 1))
h = hvector.fit_transform(headings['headingtext'])
hcount = h.toarray()
del hvector
del h

In [86]:
pvector = CountVectorizer(vocabulary=kws, ngram_range=(1, 3))
p = pvector.fit_transform(paragraphs['text'])
pcount = p.toarray()
del pvector
del p

In [87]:
sumpcount = sum(pcount)

In [88]:
sumhcount = sum(hcount)

In [89]:
kwc = {}
hweight = 5
pweight = 1
for kw in kws:
    kwc[kw] = sumhcount[kws[kw]]*hweight
    kwc[kw] += sumpcount[kws[kw]]*pweight

In [90]:
kwc['blockchain technology']/kwc['blockchain']

0.06682392306996654

In [107]:
newkws = {}

In [108]:
headings.head()

,tag,headingtext,subheading,nextheading,paragraphs
0,topic,Is Blockchain-Based Watch-to-Earn Monetization...,2,None,[]
1,h3,@ishantechIshanOnTech,None,None,[0]
2,h2,What Is Happening?,None,3,[1]
3,h2,What is Coub.com?,None,4,"[2, 3]"
4,h2,How is Coubâs Web3 Video Space Going to Work?,None,5,"[4, 5, 6, 7]"


In [109]:
for hid in headings.index:
    clear_output(wait=True)
    print(hid)
    try:
        hvector = CountVectorizer(analyzer='word', ngram_range=(1, 1))
        h = hvector.fit_transform([headings.iloc[hid]['headingtext']])
        htemp_ = hvector.vocabulary_
        htemp = list(htemp_)
        nextw = ''
        while True:

            if len(htemp)==0:
                break

            if len(nextw)==0:
                if htemp[0] in kws:
                    nextw = htemp.pop(0)
                    continue
                else:
                    htemp.pop(0)
                    continue
            else:
                if nextw+' '+htemp[0] in kws:
                    if kwc[nextw+' '+htemp[0]]/kwc[nextw]>0.05:
                        nextw += (' '+htemp.pop(0))
                        continue
                    else:
                        if nextw in newkws:
                            newkws[nextw] += 1
                        else:
                            newkws[nextw] = 1
                        htemp.pop(0)
                        nextw = ''
                        continue
                else:
                    if nextw in newkws:
                        newkws[nextw] += 1
                    else:
                        newkws[nextw] = 1
                    htemp.pop(0)
                    nextw = ''
                    continue
    except ValueError:
        continue

64682


In [110]:
len(newkws)

4126

In [111]:
newkws['proof of']

89

In [113]:
kws['proof of work']

34